Pablo Jimeno - UPV/EHU 2016

In [ ]:
import os.path
import shutil
import urllib
import pickle

import numpy as np
import numpy.ma as ma

import scipy.ndimage as ndimage
import scipy.integrate as integrate
from scipy.optimize import curve_fit, minimize
from scipy import fftpack
from scipy import optimize, stats

import matplotlib.pyplot as plt
import matplotlib
import matplotlib.patches as patches
from matplotlib.colors import LogNorm

import astropy.units as u
from astropy.io import fits
from astropy.cosmology import FlatLambdaCDM
from astropy.coordinates import SkyCoord  # High-level coordinates
from astropy.coordinates import ICRS, Galactic, FK4, FK5  # Low-level frames
from astropy.coordinates import Angle, Latitude, Longitude  # Angles
from astropy.convolution import convolve, convolve_fft, Gaussian2DKernel, Tophat2DKernel
from astropy.visualization import hist
import astropy.constants as const

import healpy as hp
from healpy.pixelfunc import ang2pix

from IPython.display import clear_output


# Some constants...
planck_h = const.h.cgs.value # [erg s]
k_b = const.k_B.cgs.value # [erg / K]
c = const.c.cgs.value # [cm / s]

## Styling...
fontsize = 9
matplotlib.rcParams['axes.titlesize']  = fontsize
matplotlib.rcParams['axes.labelsize']  = fontsize
matplotlib.rcParams['xtick.labelsize'] = fontsize
matplotlib.rcParams['ytick.labelsize'] = fontsize
matplotlib.rcParams['legend.fontsize'] = fontsize
    
cat_dir  = '../Catalogues'
data_dir = '../Data_files'
raw_dir  = '../Raw_data'
img_dir  = '../Images'
plot_dir = '../Plots'
dra_dir  = '/home/pablo/Paper_Draft_SZ/Images'

%run 'Utilities_objects.ipynb'
%run 'Utilities_catalogues.ipynb'
%run 'Utilities_images.ipynb'
%run 'Utilities_satellites.ipynb'

In [ ]:
def create_dir(directory):
    
    if not os.path.exists(directory):
        os.makedirs(directory)

dir_list = [
    cat_dir,
    '%s/Original_redMaPPer_DR8' % cat_dir,
    data_dir,
    '%s/Planck_patchs' % data_dir,
    '%s/Planck_stacked_maps' % data_dir,
    '%s/SZ_values' % data_dir,
    '%s/Temp_files' % data_dir,
    raw_dir,
    '%s/Planck/FITS_maps' % raw_dir,
    '%s/WISE' % raw_dir,
    img_dir,
    '%s/Planck/Stacked' % img_dir,
    '%s/Planck/PNG' % img_dir,
    plot_dir
]

for project_dir in dir_list:
    create_dir(project_dir)    

In [ ]:
#-----------------------------------------------------------------------------------------
def save_figure(fig, fig_name):
    
    fig.savefig('%s/PNG/%s.png' % (dra_dir, fig_name), bbox_inches='tight', pad_inches=0.10)
    fig.savefig('%s/%s.pdf' % (dra_dir, fig_name), bbox_inches='tight', pad_inches=0.10)
    fig.savefig('%s/%s.eps' % (dra_dir, fig_name), bbox_inches='tight', pad_inches=0.10)
        
    plt.close()
    
    print 'Plot saved in %s' % dra_dir
    
    
#-------------------------------------------------------------------
def sub_tag(sub_meta):
    
    sub_tag = '[%03d_%03d_%.3f_%.3f]' % (sub_meta[0], sub_meta[1], sub_meta[2], sub_meta[3])
    
    return sub_tag